In [2]:
import os
from dotenv import load_dotenv
import openai
# Load environment variables from .env file
from autogen import  AssistantAgent
from autogen.agentchat.contrib.llamaindex_conversable_agent import LLamaIndexConversableAgent

from dotenv import load_dotenv

from autogen import ConversableAgent, UserProxyAgent
from autogen.agentchat.contrib.capabilities import teachability
from autogen import Cache
from autogen.agentchat.contrib.capabilities.text_compressors import LLMLingua
from autogen.agentchat.contrib.capabilities.transforms import TextMessageCompressor
from autogen.agentchat.contrib.capabilities import transform_messages
import autogen
from autogen import ConversableAgent, UserProxyAgent
from typing import Literal, Annotated
load_dotenv()

openai_api_key = os.getenv("OPENAI_API_KEY")


In [4]:
config_list = [
    {
        "model":"gpt-4",
        "temperature":0.9,
        "api_key":openai_api_key
    }
]

llm_config = {
    "config_list": config_list,
    "timeout": 120,
}

chatbot = autogen.AssistantAgent(
    name="chatbot",
    system_message="For currency exchange tasks, only use the functions you have been provided with. Reply TERMINATE when the task is done.",
    llm_config=llm_config,
)

# create a UserProxyAgent instance named "user_proxy"
user_proxy = autogen.ConversableAgent(
    name="Human",
    llm_config=False,
    human_input_mode='ALWAYS',
    is_termination_msg=lambda msg: "good bye" in msg['content'] or None
)


CurrencySymbol = Literal["USD", "EUR"]


def exchange_rate(base_currency: CurrencySymbol, quote_currency: CurrencySymbol) -> float:
    if base_currency == quote_currency:
        return 1.0
    elif base_currency == "USD" and quote_currency == "EUR":
        return 1 / 1.1
    elif base_currency == "EUR" and quote_currency == "USD":
        return 1.1
    else:
        raise ValueError(f"Unknown currencies {base_currency}, {quote_currency}")


@user_proxy.register_for_execution()
@chatbot.register_for_llm(description="Currency exchange calculator.")
def currency_calculator(
    base_amount: Annotated[float, "Amount of currency in base_currency"],
    base_currency: Annotated[CurrencySymbol, "Base currency"] = "USD",
    quote_currency: Annotated[CurrencySymbol, "Quote currency"] = "EUR",
) -> str:
    quote_amount = exchange_rate(base_currency, quote_currency) * base_amount
    return f"{quote_amount} {quote_currency}"

In [10]:
user_proxy.function_map['currency_calculator']._origin

<function __main__.currency_calculator(base_amount: Annotated[float, <autogen.tools.dependency_injection.Field object at 0x00000232D18E2570>], base_currency: Annotated[Literal['USD', 'EUR'], <autogen.tools.dependency_injection.Field object at 0x00000232D62534A0>] = 'USD', quote_currency: Annotated[Literal['USD', 'EUR'], <autogen.tools.dependency_injection.Field object at 0x00000232D62537D0>] = 'EUR') -> Any>

In [9]:
with Cache.disk() as cache:
    # start the conversation
    res = user_proxy.initiate_chat(
        chatbot, message="How much is 123.45 USD in EUR?", summary_method="reflection_with_llm", cache=cache
    )

Human (to chatbot):

How much is 123.45 USD in EUR?

--------------------------------------------------------------------------------
chatbot (to Human):

***** Suggested tool call (call_GkXQJ4MbWOOcRC8Ujax7isOw): currency_calculator *****
Arguments: 
{
  "base_amount": 123.45,
  "base_currency": "USD",
  "quote_currency": "EUR"
}
************************************************************************************

--------------------------------------------------------------------------------
Human (to chatbot):

hello

--------------------------------------------------------------------------------
chatbot (to Human):

Hello! How can I assist you today?

--------------------------------------------------------------------------------
Human (to chatbot):

what is the converted rate?

--------------------------------------------------------------------------------
chatbot (to Human):

***** Suggested tool call (call_AC61AOeBQKlHgYzBheMSYM91): currency_calculator *****
Arguments: 
{
  

TypeError: argument of type 'NoneType' is not iterable